# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import re
import time
import pandas as pd
import google.generativeai as genai

In [2]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = 'Prompt2_Consistency_Check_Sentiment_Analysis_Results.csv'

# Import Data

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('../02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('../02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('../02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('../02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [4]:
# Drop rows google gemini will not process
rows_to_drop = ['PQ-2840736837']
index_to_drops = text_df[text_df['Unique_ID'].isin(rows_to_drop)].index
text_df.drop(index_to_drops, inplace=True)
print(index_to_drops)

Index([5379], dtype='int64')


In [5]:
# Sample 5 random articles from each source
# Identify the unique sources
unique_sources = text_df['Source'].unique()
sampled_df = pd.DataFrame()

# Sample 5 random rows from each source
for source in unique_sources:
    source_df = text_df[text_df['Source'] == source]
    sampled_rows = source_df.sample(n=5, random_state=42)
    sampled_df = pd.concat([sampled_df, sampled_rows], ignore_index=True)

display(sampled_df)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-3734,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN
1,Investment Research,IR-726,MRO,2022-05-21,Marathon Oil Corporation,"Stock Report | May 21, 2022 | NYSE Symbol: MRO...",NaN
2,Investment Research,IR-299,MRO,2023-06-24,Marathon Oil Corporation,"Stock Report | June 24, 2023 | NYSE Symbol: MR...",NaN
3,Investment Research,IR-2095,CVX,2021-11-21,Chevron Corporation,"Stock Report | November 20, 2021 | NYSE Symbol...",NaN
4,Investment Research,IR-4168,HES,2021-02-20,Hess Corporation,"Stock Report | February 20, 2021 | NYSE Symbol...",NaN
5,ProQuest,PQ-2540764382,XOM,2021-06-15,Exxon Stock Could Rise 40 With Big Dividend,"For most of the past year and a half, the big ...",https://www.proquest.com/newspapers/exxon-stoc...
6,ProQuest,PQ-2415718613,CVX,2020-06-23,Banking & Finance: Chevron Saudi-Kuwait Projec...,Big oil companies returned to profitability du...,https://www.proquest.com/newspapers/exchange-b...
7,ProQuest,PQ-2641736904,XOM,2022-03-23,So Much Climate Activism At Exxon Mobil,Many readers may have been surprised to see Ch...,https://www.proquest.com/newspapers/so-much-cl...
8,ProQuest,PQ-2880296882,OXY,NaN,Occidental Petroleum Stock Falls. It Lost a Po...,Occidental Petroleum shares had the worst show...,https://www.proquest.com/newspapers/occidental...
9,ProQuest,PQ-2857037564,EQNR,NaN,Norway: Speech by the Prime Minister at the op...,By Prime Minister Jonas Gahr StøreYour Royal H...,https://www.proquest.com/newspapers/norway-spe...


In [6]:
# Duplicate each row 10 times
repeated_df = sampled_df.loc[sampled_df.index.repeat(10)].reset_index(drop=True)

# Update the Unique_ID column
repeated_df['Unique_ID'] = repeated_df['Unique_ID'] + ' ' + repeated_df.groupby(repeated_df.index // 10).cumcount().add(1).astype(str)
display(repeated_df.head())
print(repeated_df.shape)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-3734 1,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN
1,Investment Research,IR-3734 2,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN
2,Investment Research,IR-3734 3,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN
3,Investment Research,IR-3734 4,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN
4,Investment Research,IR-3734 5,HES,2021-11-13,Hess Corporation,"Stock Report | November 13, 2021 | NYSE Symbol...",NaN


(250, 7)


# Find or Create Results CSV

In [7]:
# Determine if the Sentiment Analysis Results file already exists
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")

The file exists in the current directory.


In [8]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy repeated df and drop the text and headline column due to size
    empty_sentiment_df = repeated_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""
        empty_sentiment_df[category] = empty_sentiment_df[category].astype('object')

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

### NOTE: If you want to re-run the sentiment analysis, delete or archive the csv to create a blank one

# Sentiment Analysis
NOTE: Google gemini **currently** has a daily query limit of 1,500 requests per day.  As we have over 10,000 documents, the code will be designed to run over multiple days and pick up where we last left off.  After the code is run, the user will still need to manually download the csv and upload to github.

**Gemini Free Rate Limits**
*   15 RPM (requests per minute)
*   1 million TPM (tokens per minute)
*   1,500 RPD (requests per day)

In [9]:
# Set up Gemini. (API key needs to be in your environment variables)
key = 'GOOGLE_API_KEY'
GOOGLE_API_KEY = os.getenv(key)
genai.configure(api_key=GOOGLE_API_KEY)

# So it doesn't block the output
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",},]

model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safety_settings)

In [10]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

EP-210 7


In [11]:
# Helper function to get Gemini query function inputs
def get_gemini_inputs(text_df, unique_id):
    """
    Retrieves information from the DataFrame based on the unique ID and outputs company, source, headline, and text.

    Args:
        text_df (pd.DataFrame): The DataFrame containing the text data.
        unique_id (str): The unique ID to search for.

    Returns:
        tuple: A tuple containing company, source, headline, and text.
    """
    # Find the row with the specified unique ID
    row = text_df[text_df['Unique_ID'] == unique_id]

    # Extract the required information
    company = row['Ticker'].values[0]
    source = row['Source'].values[0]
    headline = row['Article Headline'].values[0]
    text = row['Article Text'].values[0]

    return company, source, headline, text

# Test function
company, source, headline, text = get_gemini_inputs(repeated_df, unique_id)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: SHEL

Source: Earnings Call Presentations

Headline: Royal Dutch Shell plc, Q3 2021 Earnings Call, Oct 28, 2021

Text:
Royal Dutch Shell plc ENXTAM:RDSA
FQ3 2021 Earnings Call Transcripts
Thursday, October 28, 2021 1:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ3 2021- -FQ4 2021- -FY 2021- -FY 2022-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized 0.70 0.53 (24.29 %) 0.92 2.45 3.02
Revenue (mm) 63399.50 60044.00 (5.29 %) 63362.65 301074.06 299662.46
Currency: USD
Consensus as of Oct-28-2021 8:36 AM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ4 2020 0.08 0.05 (37.50 %)
FQ1 2021 0.43 0.42 (2.33 %)
FQ2 2021 0.62 0.71 12.70 %
FQ3 2021 0.70 0.53 (24.29 %)
1
COPYRIGHT © 2021 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

ROYAL DUTCH SHELL PLC FQ3 2021 EARNINGS CALL | OCT 28, 2021
Presentation
Operator
Welcome to the Royal Shell 2021 Q3 Results Announcement Q&A session. Today's s

In [12]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output1:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 2:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Positive
 - Alternative Energy / Lower Carbon - Neutral
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Example Output 3:
 - Finance - Neutral
 - Production - Negative
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 4:
 - Finance - Positive
 - Production - Neutral
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
 - Environment / Regulatory / Geopolitics - Neutral
 - Alternative Energy / Lower Carbon - Negative
 - Oil Price / Natural Gas Price / Gasoline Price - Positive

Example Output 5:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test function
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of SHEL Earnings Call:

**Reasoning:**

The text emphasizes Shell's strong portfolio and position for economic recovery, with achievements in delivering leading cash flow and progress towards net-zero emissions. While acknowledging Third Point's letter, the focus remains on the positive performance of the company.

**Category Analysis:**

* **Finance:** **Positive** - The text highlights "sector-leading cash" and "progress towards becoming a net-zero emissions energy business", suggesting strong financial performance and commitment to a sustainable future.
* **Production:** **Neutral** - Production is not explicitly discussed in the provided excerpt. 
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:** **Neutral** -  No mention of specific reserves, exploration activities, or M&A transactions is made.
* **Environment / Regulatory / Geopolitics:** **Positive** - The statement mentions "progress towards becoming a net-zero emissions energy business"

In [13]:
# Function to parse text
def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        # Create a regex pattern to match the format "- Category - Sentiment"
        # The category name is escaped to handle any special characters
        pattern = rf"- {re.escape(category)} - (\w+)"

        # Search for the pattern in the text
        match = re.search(pattern, text)

        # If a match is found, extract the sentiment and add it to the results dictionary
        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

# Test function
fail_text = """
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Bad
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral
"""

sentiment_dict = parse_sentiment(response, CATEGORIES)
fail_sentiment = parse_sentiment("fail_text", CATEGORIES)
print(sentiment_dict)
print(fail_sentiment)

{'Finance': 'Positive', 'Production': 'Neutral', 'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Neutral', 'Environment / Regulatory / Geopolitics': 'Positive', 'Alternative Energy / Lower Carbon': 'Positive', 'Oil Price / Natural Gas Price / Gasoline Price': 'Neutral'}
Did not find all categories


In [14]:
# Function to update the csv
def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Find the index of the row with the specified unique ID
    row_index = df[df['Unique_ID'] == unique_id].index

    # Update the columns based on the sentiment dictionary
    for category, sentiment in sentiment_dict.items():
        df.loc[row_index, category] = sentiment

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

# Test function
update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

Row with Unique_ID 'EP-210 7' has been updated.


In [15]:
start_time = time.time()
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
count = 0
max_tries = 5

# Iterate through the CSV using the functions
while unique_id:
    retries = 0
    success = False

    # There are multiple errors that can happen, many of them simply need another try
    while retries < max_tries and not success:
        try:
            # Get gemini inputs
            company, source, headline, text = get_gemini_inputs(repeated_df, unique_id)

            # Query Gemini
            response = query_gemini(company, source, headline, text, model)

            # Parse text
            sentiment_dict = parse_sentiment(response, CATEGORIES)

            # Update the csv
            update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

            success = True

        except Exception as e:
            retries += 1
            print(f"Error encountered: {e}. Retry {retries}/{max_tries}")

            if retries >= max_tries:
                print(f"Failed to process unique_id {unique_id} after {max_tries} attempts. Stopping.")
                # Exit both loops
                unique_id = None
                break

    if not success:
        break

    # Get an update every 10 rows
    count += 1
    if count % 10 == 0:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Test print statements
    # print(unique_id)
    # print(f"Company: {company}\n")
    # print(f"Source: {source}\n")
    # print(f"Headline: {headline}\n")
    # print(f"Text:\n{text}")
    # print(response)
    # print(sentiment_dict)

Row with Unique_ID 'EP-210 8' has been updated.
Row with Unique_ID 'EP-210 9' has been updated.
Row with Unique_ID 'EP-210 10' has been updated.
Row with Unique_ID 'EP-257 1' has been updated.
Row with Unique_ID 'EP-257 2' has been updated.
Row with Unique_ID 'EP-257 3' has been updated.
Row with Unique_ID 'EP-257 4' has been updated.
Row with Unique_ID 'EP-257 5' has been updated.
Row with Unique_ID 'EP-257 6' has been updated.
Row with Unique_ID 'EP-257 7' has been updated.
Iteration: 10, Elapsed Time: 0 minutes and 56.89 seconds
Row with Unique_ID 'EP-257 8' has been updated.
Row with Unique_ID 'EP-257 9' has been updated.
Row with Unique_ID 'EP-257 10' has been updated.
Row with Unique_ID 'EP-98 1' has been updated.
Row with Unique_ID 'EP-98 2' has been updated.
Row with Unique_ID 'EP-98 3' has been updated.
Row with Unique_ID 'EP-98 4' has been updated.
Row with Unique_ID 'EP-98 5' has been updated.
Row with Unique_ID 'EP-98 6' has been updated.
Row with Unique_ID 'EP-98 7' has be